In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
if os.path.abspath('../') not in sys.path:
    sys.path.append(os.path.abspath('../'))

In [ ]:
from functools import partial
import torch

import numpy as np
from maxvolpy.maxvol import rect_maxvol
import matplotlib.pyplot as plt

In [ ]:
from maxvol_compression.sketch_matrix import RandomSums
from maxvol_compression.vmbf import EVBMF, weaken_rank
from maxvol_compression.layers import LinearMaxvol
from utils.dummy import DummyDatasetCifar10, DummyModelCifar10

### Dataset and Model

In [ ]:
BATCH_SIZE = 10

In [ ]:
model = DummyModelCifar10()
model.load_state_dict(torch.load('data/dummy.weights'))
cifar10 = DummyDatasetCifar10(batch_size=BATCH_SIZE, data_root='../data')

In [ ]:
model.eval()

In [ ]:
dataiter = iter(cifar10.testloader)
images, labels = dataiter.next()
_, predicted = torch.max(model(images), 1)

fig, axs = plt.subplots(2, 5, figsize=(15, 5))
for ax, img, lbl, pred in zip(axs.flatten(), images, labels, predicted):
    ax.imshow(np.transpose((img / 2 + 0.5).numpy(), (1,2,0)))
    ax.axis('off')
    ax.set_title(f'{cifar10.classes[lbl]}/{cifar10.classes[pred]}')
    
plt.show()

In [ ]:
correct = 0
all_ = len(cifar10.testloader) * BATCH_SIZE
for i, data in enumerate(cifar10.testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        _, predicted = torch.max(model(inputs), 1)
        correct += (labels == predicted).sum().detach().numpy()
        
print(f'accuracy: {correct / all_}')

### Computing sketch_matrix

In [ ]:
rs = RandomSums(120, 120, keep_original=True)
def update_sketchmatrix(self, input, output, alg):
    batch_size = input[0].shape[0]
    alg.update(input[0].view(batch_size, -1).cpu().numpy())
    
handle = model.fc2.register_forward_hook(
    partial(update_sketchmatrix, alg=rs))

In [ ]:
# handle.remove()

In [ ]:
%%time
for i, (batch,_) in enumerate(cifar10.trainloader, 1):
    with torch.no_grad():
        _ = model(batch)
    if i % 200 == 0: print(f'{i} batches completed')
handle.remove()

### EVBMF rank estimation

In [ ]:
_, sigma, Vt = np.linalg.svd(rs.sketch_matrix, full_matrices=False)
_, vbmf_s, _, vbmf_post = EVBMF(None, pretrained_svd=(None, sigma, Vt))

In [ ]:
vbmf_s.shape

In [ ]:
rank = weaken_rank(rank=min(*Vt.shape), extreme_rank =len(vbmf_s), weakenen_factor=1.0)
rank

### rect-maxvol

In [ ]:
V = Vt.T[:, :rank]
V.shape

In [ ]:
%%time
idxs, _ = rect_maxvol(V, maxK=int(1.7*min(*V.shape)))

In [ ]:
%%time
invSV = np.linalg.pinv(V[idxs, :])

In [ ]:
invSV.shape, V.shape

### compression

In [ ]:
model.fc1 = LinearMaxvol(model.fc1, idxs, V)

In [ ]:
model.eval()

In [ ]:
dataiter = iter(cifar10.testloader)
images, labels = dataiter.next()
_, predicted = torch.max(model(images), 1)

fig, axs = plt.subplots(2, 5, figsize=(15, 5))
for ax, img, lbl, pred in zip(axs.flatten(), images, labels, predicted):
    ax.imshow(np.transpose((img / 2 + 0.5).numpy(), (1,2,0)))
    ax.axis('off')
    ax.set_title(f'{cifar10.classes[lbl]}/{cifar10.classes[pred]}')
    
plt.show()

In [ ]:
correct = 0
all_ = len(cifar10.testloader) * BATCH_SIZE
for i, data in enumerate(cifar10.testloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    _, predicted = torch.max(model(inputs), 1)
    correct += (labels == predicted).sum().detach().numpy()
        
print(f'accuracy: {correct / all_}')